In [1]:
!pip install onnx
!pip install onnxruntime
!pip install onnx2pytorch
!pip install maraboupy
#!pip install tensorflow[and-cuda]==2.13.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 74.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 71.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 97.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

# Import the model here. Optionally, import the relu states.

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import onnx
from onnx2pytorch import ConvertModel
folder_path = '/content/drive/My Drive/oopsla_nap'
iris_onnx_path = folder_path + '/iris_model_60.onnx'  # Adjust file extension if needed
PATTERN_PATH = folder_path + '/relu_states.txt'  # Adjust file extension if needed

# iris_onnx_path = 'iris_model_60.onnx'  # Adjust file extension if needed
# PATTERN_PATH = 'relu_states_allen.txt'  # Adjust file extension if needed


Mounted at /content/drive


# Get the iris dataset, hook the activation to the relu layers "relu1", "relu2", "relu3".
# Find the states and save it as a dictionary. Each label is a key. Each value is another dict where the neuron_id is a key and the values are are the activations of different inputs. So the dictionary "states" is in the shape: {Label: {neuron_id: [list of length len(inputs)]}}.
#The function "convert_dict_to_list" converts the values of each label key to a list of list where the row is number of relus and column is number of inputs.
#In the next cell, the states are saved as list directly where the row is number of inputs and column is number of relus.


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import numpy as np


# Load the Iris dataset
data = load_iris()
X, y = data.data, data.target

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

In [4]:
import onnx
from onnx2pytorch import ConvertModel


onnx_model = onnx.load(iris_onnx_path)
model = ConvertModel(onnx_model)

activations = {}

def get_activation(name):
    def hook(model, input, output):
        activations[name] = output.detach()
    return hook

i=1
# Assuming the ReLU layers are named similarly to your PyTorch model, you can add hooks like this
for name, layer in model.named_modules():
    if isinstance(layer, torch.nn.ReLU):
        # Register the hook
        layer.register_forward_hook(get_activation(f'relu{i}'))
        i=i+1

import json
states={}
for i in range(3):
  states[i]={}

with torch.no_grad():
  for neuron in range(60):
    for idx, (inputs, labels) in enumerate(train_loader):
      for i,t in zip(inputs,labels):
        t=t.item()
        outputs = model(i.unsqueeze(0))
        out=activations[f'relu{(neuron//20)+1}'][:,neuron%20].tolist()[0]
        if not neuron in states[t]:
          states[t][neuron]=[]
        states[t][neuron].append(out)

def convert_dict_to_list(states): #coverts the values of each label to a list. states[label]=list of len(inputs) x len(number of relus)
  for label in states:
    label_list=[]
    for neuron in states[label]:
      label_list.append(states[label][neuron])
    states[label]=label_list
  return states

# len(STABLE_PATTERNS[0])#60 relus
# len(STABLE_PATTERNS[0][0])#40 inputs

/usr/local/lib/python3.10/dist-packages/onnx2pytorch/convert/layer.py:30: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  layer.weight.data = torch.from_numpy(numpy_helper.to_array(weight))


In [ ]:
with torch.no_grad():
  for neuron in range(60):
    for idx, (inputs, labels) in enumerate(train_loader):
      for i,t in zip(inputs,labels):
        t=t.item()
        outputs = model(i.unsqueeze(0))
        out=activations[f'relu{(neuron//20)+1}'][:,neuron%20].tolist()[0]
        if not neuron in states[t]:
          states[t][neuron]=[]
        states[t][neuron].append(out)

In [14]:
input=X_train_tensor[0]
i=1
j=1
layers={}
# Assuming the ReLU layers are named similarly to your PyTorch model, you can add hooks like this
for name, layer in model.named_modules():
  if isinstance(layer, torch.nn.ReLU):
    # Register the hook
    layers[f'relu{i}']=layer
    i=i+1
  if isinstance(layer, torch.nn.Linear):
    # Register the hook
    layers[f'linear{i}']=layer
    j=j+1
layers

def get_activations(input):
  layer1=model.Relu_result(model.Gemm_input(input.unsqueeze(0)))
  layer2=layers['relu2'](layers['linear2'](layer1))
  layer3=layers['relu3'](layers['linear3'](layer2))
  layer4=model.Gemm_15(layer3)
  all_relus = torch.concat([layer1[0].detach(),layer2[0].detach(),layer3[0].detach()])
  return all_relus
# ConvertModel(
#   (Gemm_input): Linear(in_features=4, out_features=20, bias=True)
#   (Relu_result): ReLU(inplace=True)
#   (Gemm_input.3): Linear(in_features=20, out_features=20, bias=True)
#   (Relu_result.3): ReLU(inplace=True)
#   (Gemm_input.7): Linear(in_features=20, out_features=20, bias=True)
#   (Relu_result.7): ReLU(inplace=True)
#   (Gemm_15): Linear(in_features=20, out_features=3, bias=True)
# )

In [66]:
subset_nap=label_naps[0].copy()#48, 55, 54
for i in range(45):
  subset_nap[i]='*'
for i in range(50,60):
  subset_nap[i]='*'
subset_nap.count(0)


3

In [12]:
label = 0
def get_class_of_label(l):
    ls = []
    for input, label in train_dataset:
        if label == l: ls.append(input)
    return ls

class_0 = get_class_of_label(label)
len(class_0)

40

In [18]:
def get_reduced_nap(indices, label):
    reduced_nap = []
    for i in indices:
        reduced_nap.append((i, label_naps[label][i]))
    return reduced_nap

def nap_tup(nap):
    ls = []
    for i, if_fire in enumerate(nap):
        if isinstance(if_fire, str): continue
        ls.append((i, if_fire))
    return ls


indices = [48]
label = 0
reduced_nap = get_reduced_nap(indices, label)
reduced_nap

[(48, 1)]

In [21]:
label = 1
class_0 = get_class_of_label(label)
class_0_nap = []
indices = [55]
reduced_nap = get_reduced_nap(indices, label)

for input in class_0:
    model(input.unsqueeze(0))
    sat = True
    for index, if_fire in reduced_nap:
        if not sat: continue

        relu_value = activations[f'relu{(index//20)+1}'][:,index%20].tolist()[0]
        # print(if_fire, relu_value)
        if if_fire and relu_value > 0:
            sat = True
        elif not if_fire and relu_value <= 0:
            sat = True
        else: sat = False
    if sat:
        class_0_nap.append(input)
        print("Added!")
len(class_0_nap)

Added!
Added!
Added!
Added!
Added!
Added!
Added!


7

In [13]:
match=0
temp=states[0]
temp1=label_naps[0]
for i in range(40):
  print(i)
  flag=True
  for neuron in range(60):
    nap=temp1[neuron]
    if nap==0:
      if not temp[neuron][i]==0.0:
        flag=False
      #print(f"nap should be 0, actual input: {temp[neuron][i]}")
    if nap==1:
      if not temp[neuron][i]>0:
        flag=False
      #print(f"nap should be 1, actual input: {temp[neuron][i]}")
  if flag:
    print("match")
    match+=1
print(f"matched {match}")

0
match
1
match
2
match
3
match
4
5
6
match
7
8
match
9
match
10
match
11
match
12
13
match
14
match
15
match
16
17
match
18
match
19
20
21
22
match
23
24
match
25
match
26
match
27
match
28
match
29
match
30
match
31
32
match
33
match
34
match
35
match
36
match
37
match
38
39
match
matched 29


In [114]:
label_naps[0]

[1,
 1,
 '*',
 1,
 0,
 '*',
 '*',
 0,
 1,
 '*',
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 '*',
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 '*',
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0]

In [16]:
t=0
neuron=0
match=0
index=-1
for input, target in zip(X_train_tensor, y_train_tensor):
  if target==t:
    index+=1
    relus=get_activations(input)
    flag=True
    for neuron in range(60):
      if label_naps[0][neuron]==0:
        if not relus[neuron]==0:
          flag=False
      if label_naps[0][neuron]==1:
        if not relus[neuron]>0:
          flag=False
    if flag:
      match+=1
      print(index)
print(match)

0
2
3
4
5
6
7
8
9
10
11
12
13
15
16
17
18
19
20
21
22
23
24
26
27
28
29
30
31
32
34
35
36
37
38
39
36


In [17]:
t=0
neuron=0
match=0
index=-1
for input, target in zip(X_train_tensor, y_train_tensor):
  if target==t:
    index+=1
    model(input.unsqueeze(0))
    #print(activations
    #relus=get_activations(input)
    flag=True
    for neuron in range(60):
      if label_naps[0][neuron]==0:
        if not activations[f'relu{(neuron//20)+1}'][:,neuron%20].tolist()[0]==0:
          flag=False
      if label_naps[0][neuron]==1:
        if not activations[f'relu{(neuron//20)+1}'][:,neuron%20].tolist()[0]>0:
          flag=False
    if flag:
      match+=1
      print(f"added {index}")


print(match)
    #print(activations[f'relu{(neuron//20)+1}'][:,neuron%20].tolist()[0])


added 0
added 2
added 3
added 4
added 5
added 6
added 7
added 8
added 9
added 10
added 11
added 12
added 13
added 15
added 16
added 17
added 18
added 19
added 20
added 21
added 22
added 23
added 24
added 26
added 27
added 28
added 29
added 30
added 31
added 32
added 34
added 35
added 36
added 37
added 38
added 39
36


In [15]:
activations["relu1"][0][0].tolist()

0.5013340711593628

find states as list (row: inputs, column: relus)


In [ ]:
# import json

# states={}
# for i in range(3):
#   states[i]=[] #{}

# with torch.no_grad():
#   for idx, (inputs, labels) in enumerate(train_loader):
#     for i,t in zip(inputs,labels):
#       relu_states=[]
#       t=t.item()
#       outputs = model(i.unsqueeze(0))
#       for neuron in range(60):
#         out=activations[f'relu{(neuron//20)+1}'][:,neuron%20].tolist()[0]
#         relu_states.append(out)
#         # if not t in states:
#         #   states[t]=[]
#       states[t].append(relu_states)

# with open(f"iris_relu_states", "w") as fp:
#   json.dump(states, fp)
# states

In [19]:
from maraboupy import Marabou, MarabouCore, MarabouUtils
import json
from typing import Tuple, List
import logging
import pandas

def init_network():
  network = Marabou.read_onnx(iris_onnx_path)
  return network

EPSILON = 0.5
MAX_TIME = 30 #in seconds
M_OPTIONS: MarabouCore.Options = Marabou.createOptions(verbosity=1, numWorkers=1, timeoutInSeconds=MAX_TIME)

def convert_keys_to_int(x):
    if isinstance(x, dict):
        return {int(k) if k.isdigit() else k: convert_keys_to_int(v) for k, v in x.items()}
    return x

def parse_raw_idx(raw_idx: int) -> Tuple[int, int, int]:
    n_relus = 20
    offset = 7
    layer = raw_idx // n_relus
    idx = raw_idx % n_relus
    marabou_idx = 2*n_relus*layer + idx + offset
    return layer, idx, marabou_idx

loc = 0.5
radus = 0.5
non_restricted_dim = []

Instructions for updating:
non-resource variables are not supported in the long term


if importing a existing relu states file, uncomment the following cell. Otherwise, change the name so that ***STABLE_PATTERNS=states. *** Because later code uses STABLE_PATTERNS as the variable name for the relu activations to determine whether it's open or closed.

In [20]:
with open(PATTERN_PATH, "r") as f:
  STABLE_PATTERNS = json.load(f)
  STABLE_PATTERNS=convert_keys_to_int(STABLE_PATTERNS)

In [21]:
STABLE_PATTERNS=states

find nap given states as dict. Can be commented out if not necessary. But basically it takes the activations and delta and convert it to a nap for eachlabel in the form of [0,1,0, ... *, 0,1]

In [54]:
STABLE_PATTERNS=states
delta=0.9

In [60]:
STABLE_PATTERNS[0][2]

[0.0,
 0.2269478440284729,
 0.12910127639770508,
 0.04803657531738281,
 0.0,
 0.1212189793586731,
 0.0,
 0.0,
 0.0,
 0.4165295958518982,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.37415599822998047,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.014735877513885498,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.6656107306480408]

In [5]:
STABLE_PATTERNS=states#[label][relus].count(0)

In [6]:
label_naps={}
delta=0.9
for label in STABLE_PATTERNS:
  for relus in STABLE_PATTERNS[label]:
    if ((len(STABLE_PATTERNS[label][relus])-STABLE_PATTERNS[label][relus].count(0))/len(STABLE_PATTERNS[label][relus]))>=delta:
      if not label in label_naps:
        label_naps[label]=[]
      label_naps[label].append(1)
    elif ((len(STABLE_PATTERNS[label][relus])-STABLE_PATTERNS[label][relus].count(0))/len(STABLE_PATTERNS[label][relus])) <=(1-delta):
      if not label in label_naps:
        label_naps[label]=[]
      label_naps[label].append(0)
    else:
      if not label in label_naps:
        label_naps[label]=[]
      label_naps[label].append('*')
label_naps

for label,values in label_naps.items():
  print(f"label {label} nap: {values.count(1)+values.count(0)}")

label 0 nap: 54
label 1 nap: 41
label 2 nap: 46


In [8]:
label_naps[0][48]

1

marabou algorithms. Can be run sequentially.

In [8]:
import numpy as np
def add_relu_constraints(network: Marabou.MarabouNetworkNNet, label_states, a=None)->Marabou.MarabouNetworkNNet:
    """
    Add stable relus constraints to the Marabou network
    """
    if a:
      for neuron,values in label_states.items(): #everything
        if a[neuron]==1: #include this relu
          layer, idx, marabou_idx = parse_raw_idx(neuron)
          #print(f'{neuron}: {values}')
          if (len(values)-values.count(0))/len(values) >= delta: #len(X_train)*0.5:  len(values)
              constraint = MarabouUtils.Equation(MarabouCore.Equation.GE)
              constraint.addAddend(1, marabou_idx)
              constraint.setScalar(0.001)
              network.addEquation(constraint)
          elif (len(values)-values.count(0))/len(values) <= 1-delta:
              constraint = MarabouUtils.Equation(MarabouCore.Equation.LE)
              constraint.addAddend(1, marabou_idx)
              constraint.setScalar(-0.001)
              network.addEquation(constraint)

    else:
      for neuron,values in label_states.items():
          layer, idx, marabou_idx = parse_raw_idx(neuron)
          #print(f'{neuron}: {values}')
          if values.count(0) > len(values)*0.5: #len(X_train)*0.5:  len(values)
              constraint = MarabouUtils.Equation(MarabouCore.Equation.LE)
              constraint.addAddend(1, marabou_idx)
              constraint.setScalar(-0.001)
          else:
              constraint = MarabouUtils.Equation(MarabouCore.Equation.GE)
              constraint.addAddend(1, marabou_idx)
              constraint.setScalar(0.001)
          network.addEquation(constraint)
        # import pdb;pdb.set_trace()

    return network



def check_pattern(label_states, label: int, other_label: int, a=None)->Tuple[str, int]: #relu_check_list: List[int], relu_val: List[int]
    """
    In ACAS, the prediction is the label with smallest value.
    So we check that label - other_label < 0 forall input
    by finding assignments for label - other_label >=0
    """
    print("--------CHECK PATTERN: output_{} is always less than output_{} ? --------".format(label, other_label))
    network = init_network()
    network = add_relu_constraints(network, label_states,a) #previously commented out??
    offset = network.outputVars[0][0][0]
    for i in range(4):
      network.setLowerBound(i, 0)
      network.setUpperBound(i, 1)
    #add output constraint
    constraint = MarabouUtils.Equation(MarabouCore.Equation.GE)
    constraint.addAddend(1, other_label+offset)
    constraint.addAddend(-1, label+offset)
    constraint.setScalar(0.001)
    network.addEquation(constraint)


    #add additional bounds here
    exit_code: str
    #import pdb;pdb.set_trace()
    try:
      exit_code, vals, stats = network.solve(options=M_OPTIONS)
    except Exception as e:
      print(f"exception {e}")
      exit_code = "error"
      running_time=-1

    running_time:int = stats.getTotalTimeInMicro()

    return exit_code, running_time

def main():
    res = [[-1.]*3 for i in range(3)]
    # print(res)
    for label in range(3):
        print(f"For label {label}, check if its stable RELU pattern guarantees the output")
        for other_label in range(3):#range(10):
            if other_label == int(label):
                continue
            # relu_check_list = STABLE_PATTERNS[label]["stable_idx"]
            # relu_val = STABLE_PATTERNS[label]["val"]
            exit_code, running_time = check_pattern(STABLE_PATTERNS[label], label=int(label), other_label = other_label)
            if exit_code=="sat":
                res[int(label)][other_label] = "SAT:{}".format(running_time/10**6)
                break
            elif exit_code=="unsat":
                res[int(label)][other_label] = "UNS:{}".format(running_time/10**6)

            else:
                res[int(label)][other_label] = exit_code

    res = pandas.DataFrame(res)
    print(res)

#main()

from math import e
#a=[0,1,0,1,1,…] = some binary nap set
#theorem based on s: p(F(a) = 0) ~= (d/(d+1))^d = e^(-1) as d -> inf.
#want to optimize: p(F(a) = 0) ~= e^(−1) ~= t, gradient descend on alpha.
#Intuition: alpha高, contains 更多 relus, p(F(a))更高. 所以F(a)是1的时候，descend theta, alpha变小

all_relus=60   # |J|
n=10 # num samples
theta=1
step_size=0.1
target_prob=e**(-1)
F_count=0

def get_alpha(theta):
  return (1+e**(-theta))**(-1)

def F(a):
  exit_code="error"
  for other_label in range(3):
    if other_label == int(label):
      continue
    try:
      exit_code, running_time = check_pattern(STABLE_PATTERNS[label],
                                            label=int(label), other_label = other_label, a=a)
    except Exception as e:
      print(f"exception {e}")
    if exit_code=="sat":
      return "sat"
  return exit_code #0 means robust, good!


def sample(alpha, a_L, a_U):
  a=a_L.copy()
  for j in range(len(a_U)):
    if a_U[j]==1 and a_L[j]==0:
      a[j]= np.random.choice([0, 1], p=[1-alpha, alpha])#.reshape(n,) #turn on with probability alpha
  return a

def StatRefine(a_L,theta):
  global F_count
  print(f"alpha: {get_alpha(theta)}")
  if F(a_L)=="unsat": #or current set(a_L)size太大了及时止损 but adjust alpha now, no need to care for s. In practice, s is unknown, so we seek a mechanism for setting α without this knowledge
    return a_L
  examples=[] #should alpha be updated inside or outside the loop?
  alpha=get_alpha(theta)
  for i in range(n): # generate example
    larger_set=[1 for i in range(all_relus)]
    a_i= sample(alpha, a_L, larger_set)
    if F(a_i) == "unsat":
       F_count+=1
       F_a_i = 0
    else:
       F_count+=1
       F_a_i = 1
    theta=theta-step_size*(1-F_a_i-target_prob) #If |marabou(a_i)=unsat| is large, decrease alpha (gradient descend??)
    examples.append((a_i,F_a_i))

  j_count=[0 for i in range(all_relus)]
  for j in range(all_relus):
    if a_L[j]==0:
      for (a_i, F_a_i) in examples:
        if a_i[j]==1 and F_a_i==0:
          j_count[j]+=1

  j_max=np.array(j_count).argmax()
  a_L[j_max]=1
  return StatRefine(a_L,theta)

#run StatRefine
a=[0 for i in range(all_relus)]
#StatRefine(a,theta)

In [ ]:
main()

statrefine

In [ ]:
theta=1
label=1
a=[0 for i in range(all_relus)]
final_a=StatRefine(a,theta)
print(f"{final_a.count(1)} valid relus for label: {label} = {np.where(np.array(final_a)==1)[0].tolist()}") #varies from 1 to 7??
print(f"{F_count} calls to F")

In [2]:
F_count=0
def activecoarsen(a_U,theta):
  global F_count
  alpha=get_alpha(theta)
  if alpha<e**(-1) and F(a_U)=="unsat":#a_U.count(1)<=10:
    return a_U#scancorasen(empty,a_U)
  empty=[0 for i in range(all_relus)]
  a=sample(alpha,empty,a_U)
  if F(a)=="unsat":
    F_count+=1
    F_a_i = 0
    theta=theta-step_size*(1-F_a_i-target_prob)
    return activecoarsen(a,theta) #reduced
  else:
    F_count+=1
    F_a_i = 1
    theta=theta-step_size*(1-F_a_i-target_prob)
    return activecoarsen(a_U,theta) #try again

active coarsen

In [ ]:
theta=1
label=0
final_a=activecoarsen([1 for i in range(all_relus)],theta)
print(f"valid relu for label {label}: {final_a.count(1)}, at {np.where(np.array(final_a)==1)[0].tolist()}, calls of F: {F_count}") #varies from 1 to 7??

cscan coarsen


In [ ]:
def func(relus):
    for other_label in range(3):
        exit_code, running_time = check_pattern(relus, label=int(label), other_label=other_label)
        if exit_code == "sat":
            return False
    return True

def scancoarsen(relu_label):
    keys = list(relu_label.keys())  # Convert keys to a list to avoid runtime error while modifying the dictionary
    for k in keys:
        print(f"consider neuron={k}")
        vals = relu_label[k]
        relu_label_copy = relu_label.copy()
        relu_label_copy.pop(k)
        if func(relu_label_copy):
            relu_label.pop(k)
            print(relu_label.keys())
            continue
        else:
            relu_label[k] = vals
    return list(relu_label.keys())  # Return the remaining keys

# # Assuming STABLE_PATTERNS is a list of dictionaries
# for label in range(1):
#     relu_label = STABLE_PATTERNS[label].copy()
#     remaining_keys = scancoarsen(relu_label)
#     print(f"Remaining keys after coarsening for label {label}: {remaining_keys}")

relu_remaining={}
for label in [0,1,2]:
  relu_label = STABLE_PATTERNS[label].copy()
  remaining_keys = scancoarsen(relu_label)
  relu_remaining[label]=remaining_keys

for k,v in relu_remaining.items():
  print(f"Remaining keys after coarsening for label {k}: {v}") #wow colab ai code is really good!
#print(f"Remaining keys after coarsening for label {label}: {remaining_keys}")

# **If marabou doesn't work, try the following code to see where the algorithm crashed. **

In [ ]:
network=init_network()
offset = network.outputVars[0][0][0]
label=0
other_label=1
#add constraint
for i in range(4):
  network.setLowerBound(i, 0)
  network.setUpperBound(i, 1)
#add output constraint
#why not just add all other labels??
constraint = MarabouUtils.Equation(MarabouCore.Equation.GE)
constraint.addAddend(1, other_label+offset)
constraint.addAddend(-1, label+offset)
constraint.setScalar(0.001)
network.addEquation(constraint)

for neuron,values in STABLE_PATTERNS[label].items(): #everything
  if a[neuron]==1: #include this relu
    layer, idx, marabou_idx = parse_raw_idx(neuron)
    #print(f'{neuron}: {values}')
    if values.count(0) > len(values)*0.5: #len(X_train)*0.5:  len(values)
        constraint = MarabouUtils.Equation(MarabouCore.Equation.LE)
        constraint.addAddend(1, marabou_idx)
        constraint.setScalar(-0.001)
    else:
        constraint = MarabouUtils.Equation(MarabouCore.Equation.GE)
        constraint.addAddend(1, marabou_idx)
        constraint.setScalar(0.001)
    network.addEquation(constraint)

#import pdb;pdb.set_trace()

try:
  exit_code, vals, stats = network.solve(options=M_OPTIONS)
except Exception as e:
  print(e)
  print(exit_code)

#exit_code, vals, stats = network.solve(options=M_OPTIONS)
# print(exit_code)
# print(val)
#len(network.equList)

unsat


# Some extra code that doesn't need to be run

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import numpy as np


# Load the Iris dataset
data = load_iris()
X, y = data.data, data.target

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# Define the neural network architecture
class IrisNet(nn.Module):
    def __init__(self):
        super(IrisNet, self).__init__()
        self.fc1 = nn.Linear(4, 20)  # Input layer to hidden layer
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(20, 20) # Hidden layer to hidden layer
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(20, 20) # Hidden layer to hidden layer
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(20, 3)  # Hidden layer to output layer

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.fc4(x)
        return x

# Initialize the network
model = IrisNet()

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 100
for epoch in range(epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Print loss every 10 epochs
    if epoch % 10 == 9:
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Check if the model trains without error up to this point
# "Model trained successfully!"


Epoch 10, Loss: 0.6957077980041504
Epoch 20, Loss: 0.3569485545158386
Epoch 30, Loss: 0.06546592712402344
Epoch 40, Loss: 0.14150819182395935
Epoch 50, Loss: 0.08709496259689331
Epoch 60, Loss: 0.005371565464884043
Epoch 70, Loss: 0.03119882568717003
Epoch 80, Loss: 0.001806743093766272
Epoch 90, Loss: 0.0013030058471485972
Epoch 100, Loss: 0.003509345930069685


In [ ]:
import onnx
from onnx2pytorch import ConvertModel


# onnx_model = onnx.load("iris_model.onnx")
# model = ConvertModel(onnx_model)

activations = {}

def get_activation(name):
    def hook(model, input, output):
        activations[name] = output.detach()
    return hook

i=1
# Assuming the ReLU layers are named similarly to your PyTorch model, you can add hooks like this
for name, layer in model.named_modules():
    if isinstance(layer, torch.nn.ReLU):
        # Register the hook
        layer.register_forward_hook(get_activation(f'relu{i}'))
        i=i+1

# Now, when you run a forward pass, the hooks will store the ReLU activations
with torch.no_grad():
    output = model(X_train_tensor[0].unsqueeze(0))

# Check the captured activations
activations
# Remove hooks (to clean up)
# hook1.remove()
# hook2.remove()

{'relu1': tensor([[1.2062, 1.4942, 0.0000, 2.6629, 0.0000, 0.0000, 0.1008, 0.0000, 1.2812,
          0.4878, 1.9803, 1.7296, 2.2217, 1.8856, 0.0000, 0.0000, 1.3994, 0.0000,
          0.0000, 2.1766]]),
 'relu2': tensor([[3.5583e+00, 0.0000e+00, 4.5825e+00, 0.0000e+00, 3.0476e+00, 3.5773e+00,
          0.0000e+00, 0.0000e+00, 3.5550e+00, 3.8430e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 4.2207e+00, 3.7258e+00, 5.4662e+00,
          0.0000e+00, 1.4705e-03]]),
 'relu3': tensor([[ 0.0000,  0.0000,  0.0000,  2.0140,  0.0000,  3.8408,  0.0000,  0.0000,
           3.7716,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, 10.4726,  8.1664,
           0.0000,  0.0000,  8.5828,  0.0000]])}

In [ ]:
#accuracy:
i=0
with torch.no_grad():
  for inputs,label in zip(X_test_tensor, y_test_tensor):
          outputs = model(inputs.unsqueeze(0))
          #print(outputs)
          pred=outputs.argmax() #output not normalized
          if pred==label:
            i=i+1
print(f'{i}/{len(X_test_tensor)}')

model(X_train_tensor[0].unsqueeze(0))

onnx_file_path = "iris_model_60.onnx"
torch.onnx.export(model, torch.randn(1, 4), onnx_file_path, verbose=False)

30/30


find states:

In [ ]:
states={}
for i in range(3):
  states[i]={}

with torch.no_grad():
  for neuron in range(60):
    for idx, (inputs, labels) in enumerate(train_loader):
      for i,t in zip(inputs,labels):
        t=t.item()
        outputs = model(i.unsqueeze(0))
        out=activations[f'relu{(neuron//20)+1}'][:,neuron%20].tolist()[0]
        if not neuron in states[t]:
          states[t][neuron]=[]
        states[t][neuron].append(out)

with open("relu_states", "w") as fp:
  json.dump(states, fp)

states[0]

# states
# json
# label:{{neuron: states}, }

###### initialize marabou

In [ ]:
#start with all relu，删减直到不能verify？

In [ ]:
# network=init_network()
# network.outputVars #4,5,6
# network.inputVars #0,1,2,3
# network.reluList #(i+10) for i in range(7,37)
# network.numVars #47
# offset=7



---

